## Submit the predicted result directly 

In [ ]:
# Install required libraries
# !pip install --upgrade pip
!pip install kaggle --upgrade

# Add your PRIVATE credentials (you get them from https://www.kaggle.com > 'Account' > "Create new API token" )
# Do not use "!export KAGGLE_USERNAME= ..." OR "" around your credential
%env KAGGLE_USERNAME=qaqwsw123
%env KAGGLE_KEY=1234

# Verify
!export -p | grep KAGGLE_

In [ ]:
from fastai.vision import *
import torchvision as tv
import pandas as pd
import os
from tqdm import tqdm # to see the running process
from skimage.io import imread, imshow # to read the image from workspace
from skimage.transform import resize # resize those data to a certain size for training

In [ ]:
def read_image_name(folder,resize_hight = 224, resize_width = 224):
    
    DATA_ROOT = "../input/"
    img = np.ones((1,resize_hight,resize_width,3))
    names = []
    
    for image_name in tqdm(os.listdir(folder)):
#         print(image_name)
        name_split = image_name.split('.')
        names.append(name_split[0])
        
    names = np.asarray(names)
    print("The shape of image",img.shape)
    print("The shape of image name :", names.shape)
    
    return  names


In [ ]:
# names = read_image_name('/kaggle/input/ml-marathon-final/data/kaggle_dogcat/test')
# # Load sample submission
# test_df = pd.read_csv('/kaggle/input/ml-marathon-final/sample_submission.csv')
# test_df['ID'] = names
# test_df = test_df[["ID", "Predicted"]]
# test_df= test_df.sort_values(by=['ID'])
# test_df.head()

In [ ]:
data_path=Path('/kaggle/input/ml-marathon-final/data/kaggle_dogcat')
# test = [ImageList.from_df(df, path=test_path, cols='ID', folder=folder, suffix='.jpg') 
#                for df, folder in zip([ test_df], [ 'test'])]
path = Path('/kaggle/input/ml-marathon-final/data/kaggle_dogcat/train')
test_path = Path('/kaggle/input/ml-marathon-final/data/kaggle_dogcat/test')

In [ ]:

tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
# tfms = get_transforms(max_rotate=20, max_zoom=1.3, max_lighting=0.4, max_warp=0.4, p_affine=1., p_lighting=1.)

data = (ImageList.from_folder(path)
        .split_by_rand_pct(0.3, seed=2)
        .label_from_folder()
        .add_test_folder(test_path)
        .transform(tfms, size=224,padding_mode='reflection')
        .databunch( bs=64, device= torch.device('cuda:0'))
        .normalize(imagenet_stats)
       )

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
def _plot(i,j,ax):
        x,y = data.test_ds[i*3+j]
        x.show(ax, y=y)

plot_multi(_plot, 3, 3, figsize=(8,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
wd=1e-1
learn = cnn_learner(data, models.resnet34, metrics=error_rate,wd=wd,bn_final=True)
# learn = cnn_learner(data, models.resnet34, metrics=error_rate,bn_final=True)

## Start training


In [ ]:
# learn.fit_one_cycle(4)
learn.fit_one_cycle(3, slice(1e-2), pct_start=0.8)

In [ ]:
learn.model

In [ ]:
learn.model_dir= '/kaggle/working/'

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, max_lr=slice(1e-6,5e-5),pct_start=0.8)

In [ ]:
learn.save('stage-2')

## Make Submit

In [ ]:
# df = pd.concat(dataframes)
# df.to_csv('submission.csv', index=False)
# df.head()

In [ ]:
test_preds = learn.get_preds(DatasetType.Test)

In [ ]:
# def predict(idx):
#     pred_vals = predictions[0][idx]
#     tags = find_tags(pred_vals, 0.2)
#     print(tags)
#     img = learn.data.test_ds[idx][0]
#     return img

# def find_tags(pred, thresh):
#     classes = ""
#     for idx, val in enumerate(pred):
#         if val > thresh:
#             classes = f'{classes} {learn.data.classes[idx]}'
#     return classes.strip()

In [ ]:
test_df = pd.read_csv('/kaggle/input/ml-marathon-final/sample_submission.csv')
test_df = test_df[["ID", "Predicted"]]
test_df['Predicted'] =  test_preds[0][:,0]

name_list=[]
for i in range(len(learn.data.test_ds.x)):
    image_path = learn.data.test_ds.x.items[i]
    image_name = re.search(r'([^/]+)$', f'{image_path}')[0].replace('.jpg', '')
    name_list.append(image_name)

names = np.asarray(name_list)
test_df['ID'] = names
test_df= test_df.sort_values(by=['ID'])
csv_path ='/kaggle/working/submission.csv'
test_df.to_csv(csv_path, header=["ID", "Predicted"], index=False)

In [ ]:
test_df.head()

In [ ]:
!kaggle competitions submit -c ml-marathon-final -f submission.csv -m "Notes"

# View results
!kaggle competitions submissions -c ml-marathon-final